In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import torch
from torch_geometric.data import InMemoryDataset, Data, DataLoader
import numpy as np

In [2]:
df = pd.read_csv("dataset/yoochoose-clicks.dat")

/home/pranjal/miniconda3/envs/gcn/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [7]:
df.columns = ["session_id", "timestamp", "item_id", "category"]

In [8]:
sampled_session_id = np.random.choice(df.session_id.unique(), 1000000, replace=False)
df = df.loc[df.session_id.isin(sampled_session_id)]
df.nunique()

session_id    1000000
timestamp     3566157
item_id         35421
category          238
dtype: int64

In [9]:
df.shape

(3567065, 4)

In [10]:
grouped = df.groupby("session_id")

In [11]:
data_list = []
count = 0
for session_id, group in grouped:
    sess_item_id = LabelEncoder().fit_transform(group.item_id)
    group = group.reset_index(drop=True)
    group["sess_item_id"] = sess_item_id
    group["label"] = False
    node_features = (
        group.loc[group.session_id == session_id, ["sess_item_id", "item_id"]]
        .sort_values("sess_item_id")
        .item_id.drop_duplicates()
        .values
    )
    node_features = torch.LongTensor(node_features).unsqueeze(1)
    target_nodes = group.sess_item_id.values[1:]
    source_nodes = group.sess_item_id.values[:-1]

    edge_index = torch.tensor([source_nodes, target_nodes], dtype=torch.long)
    x = node_features

    y = torch.FloatTensor([group.label.values[0]])

    data = Data(x=x, edge_index=edge_index, y=y)
    data_list.append(data)
    count+=1

KeyboardInterrupt: 

In [19]:
l1 = torch.nn.Linear(128,128)

In [23]:
df.item_id.max() +1

643078784

In [24]:
len(df.item_id.unique())

2830

In [ ]:
item_embedding = torch.nn.Embedding(num_embeddings=df.item_id.max() +1, embedding_dim=16)

In [20]:
train_loader = DataLoader(data_list,batch_size=1024)

In [21]:
for data in train_loader:
    print(data)
    x, edge_index, batch = data.x, data.edge_index, data.batch
    x = item_embedding(x)
    x = x.squeeze(1)        

    x = F.relu(self.conv1(x, edge_index))
    op = l1(d)
    print(op)
    break

Batch(batch=[3091], edge_index=[2, 3076], x=[3091, 1], y=[1024])


AttributeError: 'Batch' object has no attribute 'dim'

In [ ]:
op = l1()